In [1]:
import src.data as data

data.update.DataUpdater.update_data()
data.update.DataUpdater.preprocess_data(True)

Update Files
Tue May 14 07:00:17 2024 : Updated ~ data\DataBase\DB_information\calendar.feather Done! Cost 0.05 Secs
Tue May 14 07:00:17 2024 : Updated ~ data\DataBase\DB_information\description.feather Done! Cost 0.06 Secs
Tue May 14 07:00:17 2024 : Updated ~ data\DataBase\DB_information\st.feather Done! Cost 0.03 Secs
Tue May 14 07:00:21 2024 : Updated ~ data\DataBase\DB_information\industry.feather Done! Cost 3.76 Secs
Tue May 14 07:00:26 2024 : Updated ~ data\DataBase\DB_information\concepts.feather Done! Cost 5.23 Secs
Tue May 14 07:00:27 2024 : Updated ~ data\DataBase\DB_models\longcl_exp\2024\longcl_exp.20240508.feather Done! Cost 0.11 Secs
Tue May 14 07:00:27 2024 : Updated ~ data\DataBase\DB_models\longcl_exp\2024\longcl_exp.20240509.feather Done! Cost 0.10 Secs
Tue May 14 07:00:27 2024 : Updated ~ data\DataBase\DB_models\longcl_exp\2024\longcl_exp.20240510.feather Done! Cost 0.10 Secs
Tue May 14 07:00:27 2024 : Updated ~ data\DataBase\DB_models\risk_exp\2024\risk_exp.20240513

In [1]:
from src.interface import Predictor

model_preds = [
    Predictor('gru_day'    , 'swalast' , 0 , 'gru_day_V0') ,
    Predictor('gruRTN_day' , 'swalast' , 0 , 'gruRTN_day_V0') ,
    Predictor('gruRES_day' , 'swalast' , 0 , 'gruRES_day_V0')
]
[md.get_df().deploy() for md in model_preds]

[{'path': 'd:\\Coding\\learndl\\learndl\\model/gru_day/0', 'hidden_dim': 32, 'seqlens': {'day': 30, '30m': 30, 'dms': 30}, 'tra_seqlens': {'hist_loss': 40}, 'dropout': 0.1, 'enc_in': True, 'enc_att': False, 'rnn_type': 'lstm', 'rnn_att': False, 'rnn_layers': 2, 'dec_mlp_layers': 2, 'num_output': 1, 'kernel_size': 3, 'hidden_as_factor': False, 'ordered_param_group': False, 'tra_num_states': 5}, {'path': 'd:\\Coding\\learndl\\learndl\\model/gru_day/1', 'hidden_dim': 64, 'seqlens': {'day': 30, '30m': 30, 'dms': 30}, 'tra_seqlens': {'hist_loss': 40}, 'dropout': 0.1, 'enc_in': True, 'enc_att': False, 'rnn_type': 'lstm', 'rnn_att': False, 'rnn_layers': 2, 'dec_mlp_layers': 2, 'num_output': 1, 'kernel_size': 3, 'hidden_as_factor': False, 'ordered_param_group': False, 'tra_num_states': 5}]
Load  2 DataBlocks......  cost 0.11 secs
Align 2 DataBlocks......  cost 0.36 secs
Pre-Norming method of [day] : {'divlast': True, 'histnorm': True}
20231201 new
[{'path': 'd:\\Coding\\learndl\\learndl\\model

[None, None, None]

In [1]:
import src.data as data
data.update.DataUpdater.preprocess_data(True)

predict is True , Data Processing start!
3 datas :['y', 'trade_day', 'trade_30m']
y blocks loading start!
labels blocks reading ret10_lag DataBase's......  cost 0.77 secs
labels blocks reading ret20_lag DataBase's......  cost 0.71 secs
labels blocks merging......  cost 0.11 secs
models blocks reading risk_exp DataBase's......  cost 3.03 secs
models blocks merging......  cost 0.41 secs
y blocks loading cost 5.03 secs
y blocks process......  cost 5.04 secs
y blocks masking......  cost 0.09 secs
y blocks saving ......  cost 0.14 secs
y blocks norming......  cost 0.00 secs
Thu May  9 09:14:53 2024 : y finished! Cost 10.30 Seconds
trade_day blocks loading start!
trade blocks reading day DataBase's......  cost 1.60 secs
trade_day blocks merging......  cost 0.00 secs
trade_day blocks loading cost 1.62 secs
trade_day blocks process......  cost 0.12 secs
trade_day blocks masking......  cost 0.07 secs
trade_day blocks saving ......  cost 0.12 secs
trade_day blocks norming......  cost 0.00 secs
T

In [1]:
import src.data as data
data.update.DataUpdater.preprocess_data(False)

IndexError: string index out of range

In [4]:
model_preds[0].get_df()

Load  2 DataBlocks......  cost 0.06 secs
Align 2 DataBlocks......  cost 0.21 secs
Pre-Norming method of [day] : {'divlast': True, 'histnorm': True}


ValueError: max() arg is an empty sequence

In [2]:
from src.interface import ModelTestor

mt = ModelTestor.new('ts_mixer')
mt.try_metrics()

Load  2 DataBlocks ...

... cost 0.22 secs
Align 2 DataBlocks ...... cost 0.21 secs
Pre-Norming method of [day] : {'divlast': True, 'histnorm': True}
x shape is torch.Size([5064, 30, 6])
y shape is torch.Size([5064, 1])
Test Forward Success
metrics :  Metrics.MetricOutput(loss=tensor(1.0688, grad_fn=<AddBackward0>), score=-0.015420470386743546, loss_item=1.068833589553833, penalty=0.0, losses=tensor(1.0688, grad_fn=<ExpBackward0>))
Test Metrics Success
